In [1]:
import sklearn
import time
import lxml
from lxml import etree
import pandas as pd

In [2]:
MAX_HREF_NUM = 500

In [3]:
import os
import chardet

def get_data(path, ispms):

    html_path = path
    files = os.listdir(html_path)
    test_datas = []
    a = time.time()


    for index, file in enumerate(files, 1):
        if '.html' not in file:
            continue
        if index % 100 == 0:
            print('{} sites have been processed correctly.'.format(index))
        file_domain = file.split('.html')[0]
        file_path = os.path.join(html_path, file)
        html_data = []
        with open(file_path, 'rb') as f:
            results = f.read()
            html = etree.HTML(results)
            if html is None:
                continue
            html_data = html.xpath('//*[@href]/text()')
            if len(html_data) >= MAX_HREF_NUM:  # 只选取前多少个带有链接的文本
                html_data = html_data[:MAX_HREF_NUM]
        if html_data == []:
            continue


        one_item = ''
        code = ''
        for item in html_data:
            code = chardet.detect(item.encode())['encoding']
            item = item.strip()
            if code is None:
                code = 'utf8'
            else:
                try:
                    item = item.encode().decode(code)
                except UnicodeDecodeError:
                    continue
            one_item += ' '
            one_item += item
        movie_turple = (file_domain, ispms, one_item)
        test_datas.append(movie_turple)
    b = time.time()
    print('time used: {}'.format(b-a))
    
    return test_datas

In [5]:
pms_path = '../splashHtmlCrawler/html/pms'
not_pms_path = '../splashHtmlCrawler/html/notpms'
pms_data = get_data(pms_path, 1)
not_pms_data = get_data(not_pms_path, 0)

100 sites have been processed correctly.
200 sites have been processed correctly.
300 sites have been processed correctly.
400 sites have been processed correctly.
500 sites have been processed correctly.
600 sites have been processed correctly.
700 sites have been processed correctly.
800 sites have been processed correctly.
900 sites have been processed correctly.
time used: 55.93348836898804
100 sites have been processed correctly.
200 sites have been processed correctly.
300 sites have been processed correctly.
400 sites have been processed correctly.
500 sites have been processed correctly.
600 sites have been processed correctly.
700 sites have been processed correctly.
800 sites have been processed correctly.
900 sites have been processed correctly.
1000 sites have been processed correctly.
1100 sites have been processed correctly.
1200 sites have been processed correctly.
1300 sites have been processed correctly.
1400 sites have been processed correctly.
time used: 49.539054155

In [6]:
all_datas = pms_data + not_pms_data

In [7]:
pd_raw_data = pd.DataFrame(all_datas)
pd_raw_data.columns = ['domain', 'target', 'text']
pd_raw_data

,domain,target,text
0,00000.host,1,è®ºååè¡¨ å ¶ä...
1,00ysw.com,1,首页 电影 连续剧 综艺 动漫 更多 > 动作片 喜剧片 爱情片 科幻片 恐怖片 剧情片 ...
2,0xo.net,1,零零零 首页 無用 归档 搜索&标签 左邻右里 应急储备 到此一游 代码高亮 VP$10 ...
3,100161.com,1,首页 电影 连续剧 综艺 动漫 资讯 专题 留言 导航 取消 看过 登录 首页 电影 连续...
4,100fyy.net,1,100分影院 电视剧 电影 动漫 综艺 播出表 首页 电视剧 电影 动漫 综艺 播出表 观...
...,...,...,...
2247,zqb.cyol.com,0,中青在线 加入收藏 新闻回顾 检索 中青论坛 广告 首页 中国青年报 打印 关闭
2248,zsb.cucn.edu.cn,0,首页 学校简介 校友风采 热点镜头 校园照片 国际交流 观看全景 招考信息 招生资讯...
2249,zw.liuxue86.com,0,作文 议论文 评论 最新更新 频道地图 九月最值得期待的院线电影 字典 九月最值得期待的院...
2250,zy3.xidian.edu.cn,0,学校首页 English 首页 书院概况 书院简介 机构设置 团队介绍 书院动态 通知公告...


In [8]:
pd_raw_data.to_csv('movie_raw_data.csv')